In [1]:
import sys
sys.path.append('../../../')
from sklearn.datasets import *
from data_tools import *
from data_tools import PandasTools as pt
from data_tools import ChartTools as ct
from data_tools import AnalyzeTools as at
from data_tools import SKLTools as st
ct.IPYTHON_MODE = True
import pandas as pd

train = pt.read_csv("train.csv")
train.set_index(['Id'], inplace=True)
test = pt.read_csv("test.csv")
test.set_index(['Id'], inplace=True)
target = np.log( train['SalePrice'])
data = train.drop('SalePrice', axis=1).append(test)

============= Schema Info ===============
Path:train.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl

In [ ]:

cols2 = ["MSZoning", "MasVnrType", "BsmtFullBath", "BsmtHalfBath", "Utilities", "Functional",
         "Electrical", "KitchenQual", "SaleType", "Exterior1st", "Exterior2nd"]
for col in cols2:
    data[col].fillna(data[col].mode()[0], inplace=True)
    
cols=["MasVnrArea", "BsmtUnfSF", "TotalBsmtSF", "GarageCars",
      "BsmtFinSF2", "BsmtFinSF1", "GarageArea"]
for col in cols:
    data[col].fillna(data[col].median(), inplace=True)

cols1 = ["MiscFeature", "Alley", "Fence", "GarageFinish", "GarageYrBlt", "GarageType", "MasVnrType"]
for col in cols1:
    data[col].fillna("Nan", inplace=True)


dummies_list = ['MSZoning', 'MSSubClass', 'Street',
                'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
                'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
                'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'Foundation', 'ExterCond', 'BsmtQual',
                'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
                'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
                'SaleType', 'SaleCondition', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'MoSold']
#'OverallQual', 'OverallCond',
std_list = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
            '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea',
            'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'YearGarage', 'YearSale', 'Year']

data = at.dummies(data, dummies_list)
data['LotFrontage'].fillna(data['LotFrontage'].median(), inplace=True)


data['YearRemodAdd'] = data['YearRemodAdd'].astype(float)
data['YearBuilt'] = data['YearBuilt'].astype(float)
data['YrSold'] = data['YrSold'].astype(float)
data['GarageYrBlt'] = data['GarageYrBlt'].astype(float)
data['Year'] = data['YearRemodAdd']-data['YearBuilt']
data['YearGarage'] = data['GarageYrBlt']-data['YearBuilt']
data['YearSale'] = data['YrSold']-data['YearBuilt']
data = at.dummies(data, ['YrSold'])
data.drop(['YearRemodAdd', 'YearBuilt',
           'GarageYrBlt'], axis=1, inplace=True)
data['YearGarage'].fillna(data['YearGarage'].median(), inplace=True)
data['GarageCars'].fillna(data['GarageCars'].median(), inplace=True)
data = at.z_scores_std(data, std_list)

data = data.astype(float)
train_data = data[:1460]
test_data = data[1460:]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

reg = RandomForestRegressor(n_estimators=800)
selected_cols = st.select_features_from_model(reg, train_data, target)

train_data_new = train_data[selected_cols]
test_data_new = test_data[selected_cols]

In [ ]:
# from sklearn.metrics import make_scorer
# scorer = make_scorer(st.root_mean_squared_log_error)
# st.regression_model_estimation(
#     reg, train_data_new, target, learning_curve_scoring=scorer)

In [ ]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=1200,
       n_jobs=-1, num_leaves=40, objective=None, random_state=None,
       reg_alpha=0.001, reg_lambda=0.002, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

st.regression_model_estimation(
    lgb, train_data_new, target )